In [1]:
import json
import pandas
import numpy as np
from gensim import models

In [2]:
model = models.Doc2Vec.load('./d2w_model/model.bin')

In [3]:
def compare_sentences(sentence_a, sentence_b, iters=50):
    s = 0
    if sentence_a == sentence_b:
        return 1.0
    for i in range(iters):
        a = model.infer_vector(sentence_a.split(), steps=10)
        b = model.infer_vector(sentence_b.split(), steps=10)
        s += np.dot(a, b)/np.linalg.norm(a)/np.linalg.norm(b)
    return s/iters

In [4]:
def take_second(elem):
    return elem[1]

In [5]:
entities = [['the united states of america', 98.8],
            ['washington', 96.6],
            ['the united states', 98.6],
            ['five', 82.5],
            ['north america', 100.0],
            ['canada', 96.0],
            ['mexico', 89.5],
            ['alaska', 100.0],
            ['hawaii', 100.0],
            ['the mid - pacific', 93.2], 
            ['the pacific ocean', 97.5],
            ['the caribbean sea', 100.0]]

In [6]:
entities.sort(key=take_second, reverse=True)
print(entities)

[['north america', 100.0], ['alaska', 100.0], ['hawaii', 100.0], ['the caribbean sea', 100.0], ['the united states of america', 98.8], ['the united states', 98.6], ['the pacific ocean', 97.5], ['washington', 96.6], ['canada', 96.0], ['the mid - pacific', 93.2], ['mexico', 89.5], ['five', 82.5]]


In [7]:
matrix_shape = 10
entities_popular = entities[:matrix_shape]
print(entities_popular)

[['north america', 100.0], ['alaska', 100.0], ['hawaii', 100.0], ['the caribbean sea', 100.0], ['the united states of america', 98.8], ['the united states', 98.6], ['the pacific ocean', 97.5], ['washington', 96.6], ['canada', 96.0], ['the mid - pacific', 93.2]]


In [8]:
relationship_matrix = np.zeros(shape=(matrix_shape, matrix_shape), dtype=float)

for i in range(matrix_shape):
    for j in range(i, matrix_shape):
        # print(compare_sentences(entities_popular[i][0], entities_popular[j][0]))
        # print(entities_popular[i][0], '--->', entities_popular[j][0])
        relationship_matrix[i, j] = compare_sentences(entities_popular[i][0], entities_popular[j][0])

In [9]:
labels = [x[0] for x in entities_popular]
# print(labels)

In [10]:
df = pandas.DataFrame(data=relationship_matrix, columns=labels, index=labels)

In [11]:
df

,north america,alaska,hawaii,the caribbean sea,the united states of america,the united states,the pacific ocean,washington,canada,the mid - pacific
north america,1.0,0.60159,0.638545,0.991475,0.993084,0.994418,0.993212,0.710439,0.697457,0.957964
alaska,0.0,1.00000,0.670335,0.598727,0.595867,0.597164,0.595802,0.540570,0.603701,0.611388
hawaii,0.0,0.00000,1.000000,0.638974,0.632868,0.636785,0.639703,0.555927,0.596242,0.639369
the caribbean sea,0.0,0.00000,0.000000,1.000000,0.990813,0.991930,0.991604,0.701840,0.694384,0.958211
the united states of america,0.0,0.00000,0.000000,0.000000,1.000000,0.993132,0.992315,0.703319,0.693599,0.957967
the united states,0.0,0.00000,0.000000,0.000000,0.000000,1.000000,0.993998,0.708864,0.691494,0.959360
the pacific ocean,0.0,0.00000,0.000000,0.000000,0.000000,0.000000,1.000000,0.704410,0.697320,0.954463
washington,0.0,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.563230,0.691904
canada,0.0,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.692806
the mid - pacific,0.0,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000


In [12]:
print(df.loc['north america'].at['alaska'])
print(df.iloc[3, 4])
print(df.index[3])
print(df.columns[9])

0.6015896689891815
0.9908133590221405
the caribbean sea
the mid - pacific


In [13]:
def check_max_repeat(ent):
    first_max = ent[0]
    result = []
    for x in ent:
        if x[1] == first_max[1]:
            result.append(x)
    return result

In [14]:
def check_max_score(ent_max, matrix, matrix_shape):
    result = [0]*len(ent_max)
    for number in range(len(ent_max)):
        sum_row = 0
        sum_col = 0
        for i in range(matrix_shape):
            sum_row += matrix[number, i]
            sum_col += matrix[i, number]
        sum_row -= 1
        sum_col -= 1
        result[number] = sum_row + sum_col
    return result.index(max(result))

In [15]:
entities_popular_max = check_max_repeat(entities_popular)
print(entities_popular_max)

[['north america', 100.0], ['alaska', 100.0], ['hawaii', 100.0], ['the caribbean sea', 100.0]]


In [16]:
parent_node_num = check_max_score(entities_popular_max, relationship_matrix, matrix_shape)
print('Index of parent node is %s' % parent_node_num)
tree = {'text': labels[parent_node_num], 'id': 0, 'nodes':[]}

Index of parent node is 0


In [17]:
def find_n_nodes(parent_node_num, df, matrix_shape, n):
    nodes = []
    for i in range(matrix_shape):
        i_row = df.iloc[parent_node_num, i]
        if (i_row != 0) and (i_row != 1):
            nodes.append([df.index[i], i_row])
        i_col = df.iloc[i, parent_node_num]
        if (i_col != 0) and (i_col != 1):
            nodes.append([df.columns[i], i_col])
    nodes.sort(key=take_second, reverse=True)
    result = [x[0] for x in nodes[:n]]
    return result

In [18]:
second_level_nodes = find_n_nodes(parent_node_num, df, matrix_shape, 5)
print(second_level_nodes)
for i in range(len(second_level_nodes)):
    tree['nodes'].append({'text': second_level_nodes[i], 'id': i+1, 'nodes':[]})
print(tree)

['the united states', 'the pacific ocean', 'the united states of america', 'the caribbean sea', 'the mid - pacific']
{'text': 'north america', 'id': 0, 'nodes': [{'text': 'the united states', 'id': 1, 'nodes': []}, {'text': 'the pacific ocean', 'id': 2, 'nodes': []}, {'text': 'the united states of america', 'id': 3, 'nodes': []}, {'text': 'the caribbean sea', 'id': 4, 'nodes': []}, {'text': 'the mid - pacific', 'id': 5, 'nodes': []}]}


In [19]:
def find_rest_nodes(second_level_nodes, df, matrix_shape, labels, tree, parent_node):
    result = []
    for node in labels:
        if (node not in second_level_nodes) and node != parent_node:
            nodes_list = []
            node_num = labels.index(node)
            for i in range(matrix_shape):
                i_row = df.iloc[node_num, i]
                if ((i_row != 0) and (i_row != 1) and (df.index[i] in second_level_nodes)):
                    nodes_list.append([df.index[i], i_row])
                i_col = df.iloc[i, node_num]
                if ((i_col != 0) and (i_col != 1) and (df.columns[i] in second_level_nodes)):
                    nodes_list.append([df.columns[i], i_col])
            nodes_list.sort(key=take_second, reverse=True)
            result.append([nodes_list[0][0], node])
    return result

In [20]:
third_level_nodes = find_rest_nodes(second_level_nodes, df, matrix_shape, labels, tree, labels[parent_node_num])
print(third_level_nodes)

[['the mid - pacific', 'alaska'], ['the pacific ocean', 'hawaii'], ['the united states', 'washington'], ['the pacific ocean', 'canada']]


In [21]:
idx = len(second_level_nodes) + 1
for node in third_level_nodes:
    for x in tree['nodes']:
        if x['text'] == node[0]:
            x['nodes'].append({'text': node[1], 'id': idx, 'nodes':[]})
            idx += 1

In [22]:
output = json.dumps(tree, indent=2)
line_list = output.split("\n")
for line in line_list:
    print(line)

{
  "text": "north america",
  "id": 0,
  "nodes": [
    {
      "text": "the united states",
      "id": 1,
      "nodes": [
        {
          "text": "washington",
          "id": 8,
          "nodes": []
        }
      ]
    },
    {
      "text": "the pacific ocean",
      "id": 2,
      "nodes": [
        {
          "text": "hawaii",
          "id": 7,
          "nodes": []
        },
        {
          "text": "canada",
          "id": 9,
          "nodes": []
        }
      ]
    },
    {
      "text": "the united states of america",
      "id": 3,
      "nodes": []
    },
    {
      "text": "the caribbean sea",
      "id": 4,
      "nodes": []
    },
    {
      "text": "the mid - pacific",
      "id": 5,
      "nodes": [
        {
          "text": "alaska",
          "id": 6,
          "nodes": []
        }
      ]
    }
  ]
}
